In [10]:
import os
import json
from PIL import Image
import torchvision.transforms as transforms
import torchvision

crop_base_path = './test/crop_images'

# JSON 파일에서 bbox 데이터를 사용하여 이미지를 크롭하는 함수
def crop_images(json_file_path, image_file_path):
    cropped_images = []
    try:
        # JSON 파일 로드
        with open(json_file_path, 'r') as f:
            data = json.load(f)
        
        # 이미지 파일 열기
        img = Image.open(image_file_path)
        
        # 변환할 때 사용할 크롭 및 변환 작업 정의
        transform = transforms.Compose([
            transforms.ToTensor()
        ])
        
        # 데이터 가져오기
        annotations = data['annotations']
        categories = {category['id']: category['name'] for category in data['categories']}
        # height = 1960
        height = float(data['images'][0]['height'])
        width = float(data['images'][0]['width'])
        pl_name = data['images'][0]['pl_name']  # 이미지의 pl_name 가져오기
        
        # bbox를 사용하여 이미지 크롭 및 저장
        for annotation in annotations:
            bbox = annotation['bbox']
            
            # 이미지를 크롭합니다.
            cropped_img = img.crop((bbox[1], height - (bbox[0] + bbox[2]),bbox[1] + bbox[3], height- bbox[0]))
            cropped_img = transform(cropped_img)  # Torch 텐서로 변환
            cropped_images.append(cropped_img)
            
            # 어노테이션 ID에 해당하는 카테고리 이름 가져오기
            category_id = annotation['category_id']
            category_name = categories[category_id]


            
            # 카테고리 이름에 해당하는 폴더 생성
            category_path = os.path.join(crop_base_path,pl_name, category_name)
            if not os.path.exists(category_path):
                os.makedirs(category_path)
            
            # 이미지 저장
            file_name = os.path.splitext(os.path.basename(image_file_path))[0] + '_' + str(annotation['id']) + '.jpg'
            save_path = os.path.join(category_path, file_name)
            torchvision.utils.save_image(cropped_img, save_path)
            print(f"이미지 저장 완료: {save_path}")

    except Exception as e:
        print(f"이미지 처리 중 오류 발생: {image_file_path}")
        print(e)

    return cropped_images

In [11]:
# 폴더 경로
folder_path = "./test/Validation"

# 폴더 내의 모든 이미지 파일 및 JSON 파일 목록 가져오기
image_files = []
json_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith((".jpg", ".jpeg", ".png")):
            image_files.append(os.path.join(root, file))
        elif file.endswith(".json"):
            json_files.append(os.path.join(root, file))

# 전체 파일 수
total_files = len(image_files)
processed_files = 0

# 이미지 파일과 JSON 파일 이름으로 매칭하여 처리
for image_file_path in image_files:
    # 이미지 파일 이름에서 확장자를 제외한 부분 가져오기
    image_name = os.path.splitext(os.path.basename(image_file_path))[0]
    
    # 해당 이미지 파일과 매칭되는 JSON 파일 찾기
    matching_json_files = [json_file for json_file in json_files if os.path.splitext(os.path.basename(json_file))[0] == image_name]
    
    # 매칭되는 JSON 파일이 있을 경우에만 처리
    if matching_json_files:
        json_file_path = matching_json_files[0]  # 첫 번째 매칭되는 JSON 파일 선택
        
        # 함수 호출하여 이미지 크롭
        cropped_images = crop_images(json_file_path, image_file_path)
        
        # 결과 출력
        print(f"매칭되는 이미지 및 JSON 파일: {os.path.basename(image_file_path)}, {os.path.basename(json_file_path)}")
        print(f"크롭된 이미지 수: {len(cropped_images)}")
    else:
        print(f"매칭되는 JSON 파일이 없습니다: {os.path.basename(image_file_path)}")

     # 진행률 출력
    processed_files += 1
    print(f"진행률: {processed_files}/{total_files}")

이미지 저장 완료: ./test/crop_images\장미\줄기\07_20201028_567314_0.jpg
이미지 저장 완료: ./test/crop_images\장미\줄기\07_20201028_567314_1.jpg
이미지 저장 완료: ./test/crop_images\장미\화방\07_20201028_567314_2.jpg
매칭되는 이미지 및 JSON 파일: 07_20201028_567314.jpg, 07_20201028_567314.json
크롭된 이미지 수: 3
진행률: 1/1567
이미지 저장 완료: ./test/crop_images\장미\화방\07_20201028_567302_0.jpg
매칭되는 이미지 및 JSON 파일: 07_20201028_567302.jpg, 07_20201028_567302.json
크롭된 이미지 수: 1
진행률: 2/1567
이미지 저장 완료: ./test/crop_images\장미\화방\07_20201028_567303_0.jpg
매칭되는 이미지 및 JSON 파일: 07_20201028_567303.jpg, 07_20201028_567303.json
크롭된 이미지 수: 1
진행률: 3/1567
이미지 저장 완료: ./test/crop_images\장미\화방\07_20201028_567304_0.jpg
매칭되는 이미지 및 JSON 파일: 07_20201028_567304.jpg, 07_20201028_567304.json
크롭된 이미지 수: 1
진행률: 4/1567
이미지 저장 완료: ./test/crop_images\장미\화방\07_20201028_567305_0.jpg
매칭되는 이미지 및 JSON 파일: 07_20201028_567305.jpg, 07_20201028_567305.json
크롭된 이미지 수: 1
진행률: 5/1567
이미지 저장 완료: ./test/crop_images\장미\화방\07_20201028_567306_0.jpg
매칭되는 이미지 및 JSON 파일: 07_20201028_567306.jpg, 07_